In [0]:
!pip install --user emlearn

In [1]:
from google.colab import files
uploaded = files.upload()

Saving JaipurRawData3.csv to JaipurRawData3.csv


In [0]:
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import re

Reading Dataset

In [0]:
dataset = pd.read_csv('JaipurRawData3.csv')
X = dataset.iloc[:, 1:6]
Y = dataset.iloc[:, 12].values

In [4]:
dataset.columns

Index(['date', 'meantempm', 'meandewptm', 'meanpressurem', 'maxhumidity',
       'minhumidity', 'maxtempm', 'mintempm', 'maxdewptm', 'mindewptm',
       'maxpressurem', 'minpressurem', 'precipm'],
      dtype='object')

Converting all values of rainfall greater than 0 to 1

In [0]:
Y[Y>0] = 1
#np.sum(Y[Y==1]) # to check number of values

Adding month to dataset after one hot encoding

In [0]:
month = []
date = dataset.iloc[:, 0].values
for i in range(len(date)):
  day = date[i]
  month.append(int(re.split('-', day)[1]))
month = pd.DataFrame(month)

In [0]:
onehotencoder = OneHotEncoder()
month_one_hot = pd.DataFrame(onehotencoder.fit_transform(month).toarray()[:, :-1])

Joining the two dataframes

In [0]:
X = pd.concat([month_one_hot, X], axis=1).values

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)
sc_x = StandardScaler()
X_train = sc_x.fit_transform(X_train)
X_test = sc_x.transform(X_test)

In [0]:
def models(choice):

  if choice == 0:
    import keras
    from keras.models import Sequential
    from keras.layers import Dense

    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 16))
    classifier.add(Dense(units = 12, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)
    return classifier

  if choice == 1:
    from sklearn.ensemble import RandomForestClassifier
    estimator = RandomForestClassifier(n_estimators=10, max_depth=10)
    estimator.fit(X_train, y_train)
    return estimator

  if choice == 2:
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    return gnb

  if choice == 3:
    from sklearn.svm import SVC
    svmclassifier = SVC()
    svmclassifier.fit(X_train, y_train)
    return svmclassifier

  if choice == 4:
    from sklearn.neighbors import KNeighborsClassifier
    neigh = KNeighborsClassifier(n_neighbors=3)
    neigh.fit(X_train, y_train)
    return neigh

In [0]:
from sklearn.metrics import classification_report, confusion_matrix

for choice in range(5):
  classifier = models(choice)
  y_pred = classifier.predict(X_test)
  if choice == 0:
    y_pred = (y_pred > 0.5)
  print("For choice", choice)
  print(confusion_matrix(y_test,y_pred))
  print(classification_report(y_test,y_pred))

Using TensorFlow backend.







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100





543/543 [==============================] - 11s 20ms/step - loss: 0.6811 - acc: 0.8545
Epoch 2/100
543/543 [==============================] - 0s 335us/step - loss: 0.5925 - acc: 0.8564
Epoch 3/100
543/543 [==============================] - 0s 328us/step - loss: 0.3648 - acc: 0.8564
Epoch 4/100
543/543 [==============================] - 0s 356us/step - loss: 0.3010 - acc: 0.8564
Epoch 5/100
543/543 [==============================] - 0s 360us/step - loss: 0.2928 - acc: 0.8564
Epoch 6/100
543/543 [==============================] - 0s 347us/step - loss: 0.2883 - acc: 0.8564
Epoch 7/100
543/543 [==============================] - 0s 354us/step - loss: 0.2840 - acc: 0.8564
Epoch 8/100
543/543 [==============================] - 0s 351us/step - loss: 0.2804 - acc: 0.8564
Epoch 9/100
543/543 [==============================] - 0s 377us/step - loss: 0.2769 - acc: 0.8564
Epoch 10/10

In [0]:
classifier = models(4)
y_pred = classifier.predict(X_test)


In [0]:
import emlearn
cmodel = emlearn.convert(classifier, method='inline')
cmodel.save(file='decision_tree.h')